<a href="https://colab.research.google.com/github/DatacationOrg/chatbot-workshop/blob/main/Datacation_workshop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


In [ ]:
%%shell
rm -rf .git/
git config --global init.defaultBranch main
git init
git remote add -f origin https://github.com/DatacationOrg/chatbot-workshop.git
git config core.sparseCheckout true
echo "chatbot_workshop/" >> .git/info/sparse-checkout
echo "pyproject.toml" >> .git/info/sparse-checkout
echo "README.md" >> .git/info/sparse-checkout
git fetch
git reset --hard origin/main
# Install localtunnel
npm i -g localtunnel

In [ ]:
%pip install -e .

In [ ]:
!npm install -g localtunnel

In [ ]:
%%writefile app.py
import os

import chainlit as cl
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-exp")


@cl.on_message
async def on_message(message: cl.Message):
    msg = cl.Message(content="")
    async for chunk in llm.astream(
        message.content, config=dict(callbacks=[cl.LangchainCallbackHandler()])
    ):
        await msg.stream_token(chunk.content)
    await msg.send()


In [ ]:
import time
from chatbot_workshop.processes import restart, assert_all_running, show_logs


restart("chainlit")
restart("tunnel")
time.sleep(3)  # Allow for some time to properly startup
assert_all_running()


show_logs("tunnel")
print("Tunnel password: ")
!curl -s https://loca.lt/mytunnelpassword